## How many deploy notebooks are there?!
Boa version was intended to be used because of jupyter web wallet integrations.
However, for complex interactions, simulation times are not feasible, so we're rewriting all in web3py.

## Secret PK loading

# 1. Initialization

### i) Configure RPCs & deployer

In [ ]:
from dotenv import load_dotenv
import os
import boa
from eth_account import Account
from web3 import Web3
import logging
import subprocess
import time

from ABIs import createX_abi

logging.basicConfig(level=logging.INFO, format="%(asctime)s -  %(levelname)s - %(message)s")

load_dotenv()
main_chain = "sepolia"

PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")
DRPC_API_KEY = os.environ.get("DRPC_API_KEY")
ANKR_API_KEY = os.environ.get("ANKR_API_KEY")

RPCs = {
    "ethereum": "https://rpc.ankr.com/eth/" + ANKR_API_KEY,
    "arbitrum": "https://rpc.ankr.com/arbitrum/" + ANKR_API_KEY,
    "polygon": "https://rpc.ankr.com/polygon/" + ANKR_API_KEY,
    "bsc": "https://rpc.ankr.com/bsc/" + ANKR_API_KEY,
    "base": "https://rpc.ankr.com/base/" + ANKR_API_KEY,
    "avalanche": "https://rpc.ankr.com/avalanche/" + ANKR_API_KEY,
    "optimism": "https://rpc.ankr.com/optimism/" + ANKR_API_KEY,
    "fraxtal": "https://lb.drpc.org/ogrpc?network=fraxtal&dkey=" + DRPC_API_KEY,
    "gnosis": "https://rpc.ankr.com/gnosis/" + ANKR_API_KEY,
    "sonic": "https://rpc.ankr.com/sonic_mainnet/" + ANKR_API_KEY,
    "ink": "https://lb.drpc.org/ogrpc?network=ink&dkey=" + DRPC_API_KEY,
    "mp1": "https://rpc.ankr.com/corn_maizenet",  # corn
    # "fantom": "https://lb.drpc.org/ogrpc?network=fantom&dkey="+DRPC_API_KEY, # Must redeploy later with another evm
    # "xlayer": "https://rpc.ankr.com/xlayer/"+ANKR_API_KEY,  # no mcopy
    # "taiko": "https://lb.drpc.org/ogrpc?network=taiko&dkey="+DRPC_API_KEY, # no push0
}

createX_address = "0xba5Ed099633D3B313e4D5F7bdc1305d3c28ba5Ed"

state_dict = {}

deployer = Account.from_key(PRIVATE_KEY)

for key in RPCs.keys():
    state_dict[key] = {}
    state_dict[key]["rpc"] = RPCs[key]
    boa.set_network_env(RPCs[key])
    boa.env.add_account(deployer)
    state_dict[key]["boa"] = boa.env  # oops
    state_dict[key]["w3"] = Web3(Web3.HTTPProvider(RPCs[key]))
    state_dict[key]["createx"] = boa.loads_abi(createX_abi).at(createX_address)

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        logging.info(f"Working with {boa.env.eoa} on {key}, id {boa.env.evm.patch.chain_id}")
        logging.info(
            f"Chain balance is {state_dict[key]['w3'].eth.get_balance(boa.env.eoa)/1e18 :.3f} ETH"
        )
        logging.info(f"CreateX test: {state_dict[key]['createx'].computeCreate2Address(b'',b'')}")

### ii) Parse LZ deployments data (libs and endpoints)

In [ ]:
from LZDeployments import LZDeployments

lz = LZDeployments()
for chain in state_dict:
    metadata = lz.get_chain_metadata(chain)["metadata"]
    dvn_data = lz.get_chain_metadata(chain)

    # Update state dict
    state_dict[chain].update(
        {
            "eid": metadata["eid"],
            "endpoint": metadata["endpointV2"],
            "send_lib": metadata.get("sendUln302", "unavailable"),
            "receive_lib": metadata.get("receiveUln302", "unavailable"),
            "read_lib": metadata.get("readLib1002", "unavailable"),
            "dvns": dvn_data["dvns"],
        }
    )

    # Print info
    logging.info(f"LZ details for {chain}:")
    logging.info(f"Chain eID: {metadata['eid']}\nEndpoint address: {metadata['endpointV2']}")
    logging.info(f"DVNs: {len(dvn_data['dvns'])}, Read DVNs: {len(dvn_data['dvns_lzread'])}")
    logging.info(
        f"Send lib: {state_dict[chain]['send_lib']}\n"
        f"Receive lib: {state_dict[chain]['receive_lib']}\n"
        f"Read lib: {state_dict[chain]['read_lib']}\n---"
    )

# 2. Contracts deployment

### i) Deploy main view contract and block oracles 

In [ ]:
# On mainnet viewer will tell us ground truth block data
# On other chains block oracles are consensus of various messengers (LZ in this script)
guard_bytes = bytes.fromhex(boa.env.eoa[2:] + "00")
bytes_view = os.urandom(11)
bytes_oracle = os.urandom(11)

salt_view = guard_bytes + bytes_view
salt_oracle = guard_bytes + bytes_oracle

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        if key == main_chain:
            # deploying view contract on mainnet
            contract_deployer = boa.load_partial("../contracts/MainnetBlockView.vy")
            deploycode = contract_deployer.compiler_data.bytecode
            address = state_dict[key]["createx"].deployCreate2(salt_view, deploycode)
            # contract = contract_deployer(address)
            logging.info(f"Block view deployed at {address} on {key}")
            state_dict[key]["oracle"] = contract_deployer.at(address)
        else:
            # deploying block oracle on other chains
            contract_deployer = boa.load_partial("../contracts/BlockOracle.vy")
            bytecode = contract_deployer.compiler_data.bytecode
            args = boa.util.abi.abi_encode("(address)", (boa.env.eoa,))
            deploycode = bytecode + args

            address = state_dict[key]["createx"].deployCreate2(salt_oracle, deploycode)
            # contract = contract_deployer(1)  # commit threshold
            logging.info(f"Block oracle deployed at {address} on {key}")
            state_dict[key]["oracle"] = contract_deployer.at(address)

In [ ]:
# Deploy LZBlockRelay on each chain
# This contract is used to send and receive messages via LZ, including LZRead and chained broadcasting
guard_bytes = bytes.fromhex(boa.env.eoa[2:] + "00")
bytes_relay = os.urandom(11)

salt_relay = guard_bytes + bytes_relay
# salt_relay = bytes.fromhex('73241e98090042a718f7eb1af07fad27ff09a3f3002b281cb382b00f03a22004')
for key in state_dict:
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        contract_deployer = boa.load_partial("../contracts/messengers/LZBlockRelay.vy")
        bytecode = contract_deployer.compiler_data.bytecode
        args = boa.util.abi.abi_encode("(address)", (boa.env.eoa,))
        deploycode = bytecode + args
        address = state_dict[key]["createx"].deployCreate2(salt_relay, deploycode)
        contract = contract_deployer.at(address)
        print(f"LZ Messenger deployed at {contract.address} on {key}")
        state_dict[key]["block_relay"] = contract

# 3. Contracts configuration

## i) Oracles configuration

#### Initialize with all deployed peer ids and read libs

In [ ]:
# Prepare initialization parameters for block relay contracts
for key in state_dict.keys():
    if key == main_chain:
        continue

    # Basic parameters
    endpoint = state_dict[key]["endpoint"]
    default_gas_limit = 300_000
    read_channel = 4294967295 if state_dict[key]["read_lib"] != "unavailable" else 0

    # Use dict to manage unique peer relationships
    peers_dict = {}

    # 1. If read-enabled:
    # - add self as read peer
    # - add all other chains as peers for broadcasting
    if state_dict[key]["read_lib"] != "unavailable":
        peers_dict[read_channel] = state_dict[key]["block_relay"].address

        for target_key in state_dict.keys():
            if target_key != main_chain and target_key != key:
                peers_dict[state_dict[target_key]["eid"]] = state_dict[target_key][
                    "block_relay"
                ].address

    # 2. For all chains: add read-enabled chains as peers (to receive their broadcasts)
    # duplicates are managed by dict structure
    for source_key in state_dict.keys():
        if (
            source_key != main_chain
            and source_key != key
            and state_dict[source_key]["read_lib"] != "unavailable"
        ):
            peers_dict[state_dict[source_key]["eid"]] = state_dict[source_key][
                "block_relay"
            ].address

    # Convert dict to lists
    peer_eids = list(peers_dict.keys())
    peers = list(peers_dict.values())

    # Prepare lib configuration
    channels = []
    libs = []
    lib_types = []

    # Add send and receive libs
    if state_dict[key]["send_lib"] != "unavailable":
        channels.append(state_dict[key]["eid"])
        libs.append(state_dict[key]["send_lib"])
        lib_types.append(1)

    if state_dict[key]["receive_lib"] != "unavailable":
        channels.append(state_dict[key]["eid"])
        libs.append(state_dict[key]["receive_lib"])
        lib_types.append(2)

    # Add read libs if available
    if state_dict[key]["read_lib"] != "unavailable":
        channels.extend([read_channel, read_channel])
        libs.extend([state_dict[key]["read_lib"], state_dict[key]["read_lib"]])
        lib_types.extend([1, 2])

    logging.info(f"init parameters on {key}")
    logging.info(f"Peer_eids: {peer_eids}")
    logging.info(f"Peers: {peers}")
    logging.info(f"Channels: {channels}")
    logging.info(f"Libs: {libs}")
    logging.info(f"Lib types: {lib_types}")
    logging.info(f"Initializing block relay on {key}...")

    # Initialize
    relay_contract = state_dict[key]["block_relay"]
    relay_contract.initialize(
        endpoint,
        default_gas_limit,
        read_channel,
        peer_eids,
        peers,
        channels,
        libs,
        lib_types,
    )

#### Add block oracles and committer

In [ ]:
for key in state_dict.keys():
    if key == main_chain:
        continue
    relay_contract = state_dict[key]["block_relay"]

    # add block oracles to every relayer
    if relay_contract.block_oracle() == state_dict[key]["oracle"].address:
        logging.info(f"Skipping {key} - already set")
    else:
        relay_contract.set_block_oracle(state_dict[key]["oracle"].address)
        logging.info(f"Set block oracle tx on {key}")

    # now add relayer as committer to block oracle
    oracle_contract = state_dict[key]["oracle"]
    if oracle_contract.is_committer(relay_contract.address):
        logging.info(f"Skipping {key} - already a committer")
    else:
        oracle_contract.add_committer(relay_contract.address, True)
        logging.info(f"Add committer tx on {key}")

    # add mainnet view contract on read chains (as read source)
    if state_dict[key]["read_lib"] != "unavailable":
        contract = state_dict[key]["block_relay"]
        contract.set_read_config(
            True, state_dict[main_chain]["eid"], state_dict[main_chain]["oracle"].address
        )
        logging.info(f"Set read config on {key}")

## ii) LZ Configuration

### a) Send/receive libs (uln302)

In [ ]:
# # send lib
# for key in state_dict.keys():
#     if key == main_chain:
#         continue
#     with boa.swap_env(state_dict[key]["boa"]):
#         # send lib
#         state_dict[key]["block_relay"].set_lz_send_lib(
#             state_dict[key]["eid"], state_dict[key]["send_lib"]
#         )
#         logging.info(f"Added send lib: on {key}")
#         # receive lib
#         state_dict[key]["block_relay"].set_lz_receive_lib(
#             state_dict[key]["eid"], state_dict[key]["receive_lib"]
#         )
#         logging.info(f"Added receive lib: on {key}")

### b) Read libs

In [ ]:
# for key in state_dict.keys():
#     if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
#         continue
#     with boa.swap_env(state_dict[key]["boa"]):
#         w3 = state_dict[key]["w3"]
#         lzreadlib = w3.eth.contract(address=state_dict[key]["read_lib"], abi=lzreadlib_abi)
#         logging.info(
#             f"Read lib type: {lzreadlib.functions.messageLibType().call()}, version: {lzreadlib.functions.version().call()}"
#         )
#         eid_check = [4294967295, 4294967294]
#         supported_eid = None
#         for eid in eid_check:
#             support = lzreadlib.functions.isSupportedEid(eid).call()
#             logging.info(f"Supports {eid}: {support}")
#             if support and not supported_eid:
#                 supported_eid = eid
#                 break
#         # make sure read channel is supported
#         read_channel = state_dict[key]["block_relay"].LZ_READ_CHANNEL()
#         logging.info(
#             f"Relayer's {state_dict[key]['block_relay'].address} read channel on {key}: {read_channel}"
#         )
#         if supported_eid and read_channel != supported_eid:
#             logging.info(f"Setting read channel to {supported_eid}")
#             func = state_dict[key]["block_relay"].set_lz_read_channel(supported_eid)

#         # assign read lib to send and receive on read channel
#         try:
#             logging.info(
#                 f"Setting read lib (send direction) to {state_dict[key]['read_lib']} on {key}"
#             )
#             state_dict[key]["block_relay"].set_lz_send_lib(
#                 read_channel, state_dict[key]["read_lib"]
#             )
#         except Exception:
#             pass
#         try:
#             logging.info(
#                 f"Setting read lib (receive direction) to {state_dict[key]['read_lib']} on {key}"
#             )
#             state_dict[key]["block_relay"].set_lz_receive_lib(
#                 read_channel, state_dict[key]["read_lib"]
#             )
#         except Exception:
#             pass

### c) Set peers

In [ ]:
# for read_key in state_dict.keys():
#     # cycle through read-supporting chains
#     if read_key == main_chain or state_dict[read_key]["read_lib"] == "unavailable":
#         continue

#     contract_read = state_dict[read_key]["block_relay"]

#     read_eid = state_dict[read_key]["eid"]
#     read_oapp = contract_read.address
#     LZ_READ_CHANNEL = contract_read.LZ_READ_CHANNEL()
#     # set self as read peer (so read requests can be sent)
#     if contract_read.LZ_PEERS(LZ_READ_CHANNEL) != read_oapp:
#         contract_read.set_peer(LZ_READ_CHANNEL, read_oapp)
#         logging.info(f"Set peer on {read_key}")
#     else:
#         logging.info(f"Skipping {read_key} - already set")

#     ## nested cycle through all chains
#     for key in state_dict.keys():
#         if key == main_chain or key == read_key:
#             # skip main and current read chain
#             continue

#         contract_receiver = state_dict[key]["block_relay"]
#         receiver_eid = state_dict[key]["eid"]
#         receiver_oapp = contract_receiver.address
#         # add read relayer as peer
#         try:
#             contract_receiver.set_peer(read_eid, read_oapp)
#             logging.info(
#                 f"{key} - set peer {read_oapp} (eid {read_eid}) for messenger {contract_receiver.address}"
#             )
#         except Exception:
#             logging.info(f"Skipping {key} - already set")
#         # add chain to read broadcast targets
#         try:
#             contract_read.add_broadcast_target(
#                 state_dict[key]["eid"], state_dict[key]["block_relay"].address
#             )
#             logging.info(f"{read_key} - added {key} to broadcast targets")
#         except Exception:
#             logging.info(f"Skipping {read_key} - already set")
#         # set chain to peers of read relayer
#         if contract_read.LZ_PEERS(receiver_eid) != receiver_oapp:
#             contract_read.set_peer(receiver_eid, receiver_oapp)
#             logging.info(
#                 f"{read_key} - set peer {receiver_oapp} (eid {receiver_eid}) for messenger {contract_read.address}"
#             )
#         else:
#             logging.info(f"Skipping {read_key} - already set")

### d) Set broadcast destinations

In [ ]:
# Set broadcast targets
for read_key in state_dict.keys():
    if read_key == main_chain or state_dict[read_key]["read_lib"] == "unavailable":
        continue

    contract_read = state_dict[read_key]["block_relay"]
    broadcast_eids = []
    broadcast_targets = []
    # Add all chains except main and self as broadcast targets
    for target_key in state_dict.keys():
        if target_key != main_chain and target_key != read_key:
            broadcast_eids.append(state_dict[target_key]["eid"])
            broadcast_targets.append(state_dict[target_key]["block_relay"].address)
            logging.info(f"{read_key} - added {target_key} to broadcast targets")
    contract_read.add_broadcast_targets(broadcast_eids, broadcast_targets)

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [ ]:
def get_vyper_abi(filepath):
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


ABI_RELAY = get_vyper_abi("../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../contracts/BlockOracle.vy")
for key in state_dict.keys():
    if key == main_chain:
        continue
    state_dict[key]["block_relay_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["block_relay"].address, abi=ABI_RELAY
    )
    state_dict[key]["oracle_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["oracle"].address, abi=ABI_ORACLE
    )
account = Web3().eth.account.from_key(deployer.key)


def send_tx_single(w3, func, acc, value=0, gas_multiplier=2):
    tx = func.build_transaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "value": value,
        }
    )
    tx["gas"] = int(gas_multiplier * w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(w3, func, acc, value=0):
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value)
            success = True
        except Exception as e:
            if "replacement transaction underpriced" in str(e) or "nonce too low" in str(e):
                print(str(e), "Retrying...")
                success = False
                time.sleep(0.5)
            else:
                raise e
    return tx_hash

### Simple reads without broadcast

In [ ]:
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"Current block number on {main_chain}: {main_block}")
for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        logging.info(f"Skipping {key}")
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]

    # First quote read fee
    fee = contract_w3.functions.quote_read_fee().call()
    logging.info(f"Read fee: {fee} on {key}")

    # Then request read
    func = contract_w3.functions.request_block_hash([], [])
    tx_hash = send_tx(state_dict[key]["w3"], func, account, 3 * fee)
    logging.info(f"Tx: {tx_hash.hex()} on {key}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    try:
        logging.info(f"Number difference: {main_block - number}")
    except Exception:
        pass

### Now separate broadcast

In [ ]:
broadcaster = "base-sepolia"
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
receive_eids = [
    state_dict[key]["eid"] for key in state_dict.keys() if key not in [broadcaster, main_chain]
]
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
broadcast_fees = [fee for fee in broadcast_fees]
logging.info(f"LZSend fees: {broadcast_fees} on {broadcaster}")

# broadcast call
func = broadcaster_w3.functions.broadcast_latest_block(receive_eids, broadcast_fees)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, sum(broadcast_fees))
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

### Now reads with broadcast

In [ ]:
# we can quote read fee now
# let's pick one of the read chains and quote fee
broadcaster = "arbitrum-sepolia"
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
receive_eids = [
    state_dict[key]["eid"] for key in state_dict.keys() if key not in [broadcaster, main_chain]
]
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
broadcast_fees = [fee * 2 for fee in broadcast_fees]
logging.info(f"LZSend fees: {broadcast_fees} on {broadcaster}")

BROADCAST_GAS = 2_000_000
read_fee_with_broadcast = broadcaster_w3.functions.quote_read_fee(
    0, BROADCAST_GAS, sum(broadcast_fees)
).call()
logging.info(f"LZRead fee with broadcast: {read_fee_with_broadcast} on {broadcaster}")

# magic broadcast call
main_block = state_dict[main_chain]["w3"].eth.block_number
func = broadcaster_w3.functions.request_block_hash(receive_eids, broadcast_fees, 0, BROADCAST_GAS)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, read_fee_with_broadcast)
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    try:
        logging.info(f"Number difference: {main_block - number}")
    except Exception:
        pass

### Request remote lzread

In [ ]:
# Pick a read-supporting chain to request the read from
read_chain = "arbitrum-sepolia"
requesting_chain = "optimism-sepolia"

# first use read contracts to estimate read and broadcast fees
fee_multiplier = 2
read_contract = state_dict[read_chain]["block_relay_w3"]
read_fee = fee_multiplier * read_contract.functions.quote_read_fee().call()
broadcast_fee = (
    fee_multiplier
    * read_contract.functions.quote_broadcast_fees([state_dict[requesting_chain]["eid"]]).call()[0]
)
logging.info(f"Read fee: {read_fee} on {read_chain}")
logging.info(f"Broadcast fee: {broadcast_fee} on {read_chain}")

# now request remote read
requesting_contract = state_dict[requesting_chain]["block_relay_w3"]
func = requesting_contract.functions.request_remote_read(
    state_dict[read_chain]["eid"], read_fee, broadcast_fee
)
msg_value = 2 * (read_fee + broadcast_fee)
tx_hash = send_tx(state_dict[requesting_chain]["w3"], func, account, msg_value)
logging.info(f"Tx: {tx_hash.hex()} on {requesting_chain}")